# EXAMPLE CODES FOR THE EXPERIMENTAL RECONSTRUCTIONS

In [ ]:
from src.forward_models.bornapprox import MVProd, MVProd_iterK
import src.optimization as opt
from src.nn_models.unet3D import denoiser

from src.utils.stats import l2
from src.utils.misc import Scene, super_clear_cuda
from src.utils.data import load_dataset, load_json

import torch 
import numpy as np

Load the experimental data (Please refer to "format-exp-data.ipynb" for the instructions on getting the experimental data)

In [ ]:
y = load_dataset('Yarovoy','measurements')
tx_positions = load_dataset('Yarovoy','tx_positions')
rx_positions = load_dataset('Yarovoy','rx_positions')
freqs = load_dataset('Yarovoy','freqs')

Define the radar system's parameters as well as the number of voxels and the volume boundaries for each dimension

In [ ]:

RADAR_CONFIG={
    'F_MIN':4e9, # Minimum frequency to use in the frequency band (Hz)
    'F_MAX':16e9, # Maximum frequencyto use in the frequency band (Hz)
    'N_F':11, # Total number of frequency samples  

    'X_MIN':-0.15, # image cube's minimum x coordinate 
    'X_MAX':0.15, # image cube's maximum x coordinate 
    'N_X':61, # Number of voxels on the x direction

    'Y_MIN':-0.15, # image cube's minimum y coordinate
    'Y_MAX':0.15, # image cube's maximum y coordinate 
    'N_Y':61, # Number of voxels on the y direction

    'Z_MIN':0.35, # image cube's minimum z coordinate
    'Z_MAX':0.65, # image cube's maximum z coordinate 
    'N_Z':61 # Number of voxels on the z direction
}


Utility codes for experimental measurements:

In [ ]:

def slice_freqs(y,freqs,f_min,f_max,N_f,partition='min-max-idx'):
    assert torch.any(freqs == f_min)
    assert torch.any(freqs == f_max)

    if partition=='min-max-idx':
        max_idx = torch.argwhere(freqs==f_max).item()
        min_idx = torch.argwhere(freqs==f_min).item()

        slice_idxs=torch.linspace(min_idx,max_idx,N_f).to(dtype=int)
        
        return freqs[slice_idxs], y[:,:,slice_idxs,...]

    elif partition=='min-maxBound':

        bw=f_max-f_min
        df=freqs[1]-freqs[0]
        bw_floor=  df*np.floor((bw/df)/(N_f-1))*(N_f-1)
        f_max_used=f_min+bw_floor
        
        max_idx = torch.argwhere(freqs==f_max_used).item()
        min_idx = torch.argwhere(freqs==f_min).item()

        slice_idxs=torch.linspace(min_idx,max_idx,N_f).to(dtype=int)

        assert slice_idxs.size()[0]==N_f
        
        print("fmin: {:.2e}, fmax: {:.2e}, fmax-error: {:.2e}, Nf: {}".format(freqs[slice_idxs][0],freqs[slice_idxs][-1],f_max-freqs[slice_idxs][-1],freqs[slice_idxs].shape[0]))
        
        return freqs[slice_idxs], y[:,:,slice_idxs,...]

def get_scene_coords(x_min,x_max,nx,y_min,y_max,ny,z_min,z_max,nz):
    x_coords=torch.linspace(x_min,x_max,nx)
    y_coords=torch.linspace(y_min,y_max,ny)
    z_coords=torch.linspace(z_min,z_max,nz)
    return x_coords,y_coords,z_coords

class RadarSystemParams():
    def __init__(self,freqs,tx_positions,rx_positions,x_coords,y_coords,z_coords) -> None:
        self.freqs =freqs
        self.tx_positions = tx_positions
        self.rx_positions = rx_positions
        self.x_coords = x_coords
        self.y_coords = y_coords
        self.z_coords = z_coords


Slice the measurements to be used during the reconstruction:

In [ ]:
f_used,y_sampled=slice_freqs(y,freqs,f_min=RADAR_CONFIG['F_MIN'],f_max=RADAR_CONFIG['F_MAX'],N_f=RADAR_CONFIG['N_F'],partition='min-maxBound')
assert torch.all(torch.linspace(f_used.min(),f_used.max(),f_used.size()[0],dtype=torch.float64) == f_used)
print("Utilized Frequnecies:",f_used)

x_coords,y_coords,z_coords = get_scene_coords(  x_min=RADAR_CONFIG['X_MIN'],    x_max=RADAR_CONFIG['X_MAX'], nx=RADAR_CONFIG['N_X'],
                                                y_min=RADAR_CONFIG['Y_MIN'],    y_max=RADAR_CONFIG['Y_MAX'], ny=RADAR_CONFIG['N_Y'],
                                                z_min=RADAR_CONFIG['Z_MIN'],    z_max=RADAR_CONFIG['Z_MAX'], nz=RADAR_CONFIG['N_Z'])

radar_system_params = RadarSystemParams(freqs=f_used,tx_positions=tx_positions,rx_positions=rx_positions,x_coords=x_coords,y_coords=y_coords,z_coords=z_coords)

MoverN=(tx_positions.shape[0]*rx_positions.shape[0]*RADAR_CONFIG['N_F'])/(RADAR_CONFIG['N_X']*RADAR_CONFIG['N_Y']*RADAR_CONFIG['N_Z'])

del y
del freqs


Generate the forward model matrix:

In [ ]:
A = MVProd( freqs = radar_system_params.freqs, # frequency list
            tx_positions = radar_system_params.tx_positions, # transmitter positions
            rx_positions = radar_system_params.rx_positions, # receiver positions
            x_coords = radar_system_params.x_coords, # discretized x-coordinates
            y_coords = radar_system_params.y_coords, # discretized y-coordinates
            z_coords = radar_system_params.z_coords, # discretized z-coordinates
            path = '', save = False # you can set a path to save the forward model matrix 
            )

For higher resolution images you may prefer to use the following module as it allows batchwise computation of measurements by iteratively computing the response for different frequencies:

In [ ]:
# A = MVProd_iterK( freqs = radar_system_params.freqs,
#             tx_positions = radar_system_params.tx_positions,
#             rx_positions = radar_system_params.rx_positions,
#             x_coords = radar_system_params.x_coords,
#             y_coords = radar_system_params.y_coords,
#             z_coords = radar_system_params.z_coords,
#             path = 'None', save = False,
#             f_batch_size=1)


In [ ]:
print('Forward model matrix shape: ',A.A.shape)
print('100*(M/N) = ~{:.2f}'.format(100*A.A.shape[0]*A.A.shape[1]*A.A.shape[2]/(A.A.shape[3]*A.A.shape[4]*A.A.shape[5])))

As explained in the paper, $\epsilon$ is emprically set to match ~10dB measurement SNR:

In [ ]:
SNR=10 # dB
y_sampled=y_sampled
eps = (l2(y_sampled)/(10**(SNR/20))).item()
print('epsillon',eps)

Set device

In [ ]:
device = torch.device("cpu")
torch.backends.cudnn.benchmark = True
if device != "cpu":
     super_clear_cuda(10)

Experimental Measurements' scale may be different than the simulated data. We therefore need to scale the input images before and after entering the DNN:

In [ ]:
class scale_invariance_wrapper(torch.nn.Module):
    def __init__(self,net) -> None:
        super().__init__()
        self.net=net
      
    def forward(self,xn):
        x=xn[:,0:1,:,:,:]
        n=xn[:,1:2,:,:,:]

        sc = opt.maxmag3D(x)
        x = x/sc
        z  = self.net(torch.concat([x,n],dim=1))
        return z*sc

Reconstruct the scene:

In [ ]:
with torch.no_grad():

    model_state_dict= torch.load('trained_model/base_model.pt')

    denoiser_config=load_json('trained_model/info.json')['arch']
    denoiser_net = denoiser(**denoiser_config)
    denoiser_net.load_state_dict(model_state_dict)

    denoiser_net = scale_invariance_wrapper(denoiser_net) # make the dnn scale invariant

    solver = opt.NNCG_CSALSA(forward_model=A, denoiser_net=denoiser_net, blind=False, noise_level_map='std', cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # you can use other denoising networks as well,
    # if you plan to use a blind denoiser set "blind" to True, 
    # if your noise level map is the variance of the noise, set "noise_level_map" to 'var'
    # cg_max_iter is the maximum number of conjugate gradient iterations
    # cg_th is the minimum relative improvement on the l2 norm before terminating the conjugate gradient iterations (cg_max_iter has priority over cg_th)
    # you should always use zero_phase=False. If true, it enforces real-positive values on reflectivity distribution, which is an incorrect assumption
    # by setting record to True, you can store the values of the data fidelity and regularization terms throughout the iterations (may be useful for debugging)


    # Other solvers:
    # solver = opt.TVCG_CSALSA(forward_model=A, chambolle_iter=5, cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # solver = opt.l1CG_CSALSA(forward_model=A, cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # solver = opt.BackProjection(radar_system_params=radar_system_params)
    # solver = opt.KirchhoffMigration(radar_system_params=radar_system_params,compansate_limited_aparture=False)
    
    solver.to(device)
    solver.clear_history()

    kwargs = {}

    # set the algorithm parameters
    kwargs['y'] = y_sampled.to(device)
    kwargs['eps'] = eps # epsillon parameter in the paper
    kwargs['tqdm_active'] = True # Set to True if you want to see an iteration counter
    kwargs['K'] = 500 # Kappa parameter in the paper
    kwargs['noise_var'] = 7e-2 # alpha parameter in the paper
    kwargs['max_iter']=30

    # run the solver
    solver.forward(**kwargs)
    
    # get estimates (estimates are returned as normalized magnitues, you can access complex valued reconstructions by "solver.x")
    est = solver.get_estimate().cpu()

Show the reconstructed reflectivity magnitudes

In [ ]:
scene=Scene()
scene.show_experimental(x=est,radar_system_params=radar_system_params)